In [1]:
import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve,  roc_auc_score

# plt.style.use('seaborn-colorblind')
# %matplotlib inline
#from feature_cleaning import rare_values as ra

## Load Dataset

In [2]:
use_cols = [
    'Pclass', 'Sex', 'Age', 'Fare', 'SibSp',
    'Survived'
]

data = pd.read_csv('./data/titanic.csv', usecols=use_cols)


In [3]:
data.head(3)

,Survived,Pclass,Sex,Age,SibSp,Fare
0,0,3,male,22.0,1,7.2500
1,1,1,female,38.0,1,71.2833
2,1,3,female,26.0,0,7.9250


In [4]:
# Note that we include target variable in the X_train 
# because we need it to supervise our discretization
# this is not the standard way of using train-test-split
X_train, X_test, y_train, y_test = train_test_split(data, data.Survived, test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((623, 6), (268, 6))

## Polynomial Expansion

generate a new feature set consisting of all polynomial combinations of the features with degree less than or equal to the specified degree

# Transformación con `PolynomialFeatures` (Grado 2)

Este código crea **combinaciones polinómicas** entre las variables `'Pclass'` y `'SibSp'` del dataset `X_train`. El objetivo es capturar **relaciones no lineales** que un modelo lineal no puede detectar por sí solo.


## ¿Qué es una relación lineal?

Una **relación lineal** significa que al aumentar una variable, el cambio en la otra es proporcional. Por ejemplo:

> `precio = 10 × cantidad` → relación perfectamente lineal

Pero muchas veces los datos reales tienen relaciones **no lineales**, como:

> `precio = cantidad²` o `precio = cantidad × demanda`

Estas relaciones más complejas **no pueden ser captadas por modelos lineales**... a menos que transformemos las variables.


In [5]:
# create polynomial combinations of feature 'Pclass','SibSp' with degree 2
from sklearn.preprocessing import PolynomialFeatures
pf = PolynomialFeatures(degree=2,include_bias=False).fit(X_train[['Pclass','SibSp']])
tmp = pf.transform(X_train[['Pclass','SibSp']])
X_train_copy = pd.DataFrame(tmp,columns=pf.get_feature_names_out(['Pclass','SibSp']))
print(X_train_copy.head(6))

   Pclass  SibSp  Pclass^2  Pclass SibSp  SibSp^2
0     1.0    0.0       1.0           0.0      0.0
1     1.0    1.0       1.0           1.0      1.0
2     3.0    5.0       9.0          15.0     25.0
3     1.0    0.0       1.0           0.0      0.0
4     3.0    1.0       9.0           3.0      1.0
5     2.0    1.0       4.0           2.0      1.0


## Feature Learning by Trees

# GBDT + Logistic Regression: Generación de Features y Clasificación

Este enfoque combina **árboles de decisión (GBDT)** con **regresión logística**, generando nuevas columnas a partir de los nodos hoja en los que cae cada muestra.

¿Por qué usar GBDT + LR?
Aprovechas el poder predictivo de GBDT.

Aprovechas la eficiencia y simplicidad de la regresión logística.

Muy útil en sistemas de recomendación, scoring, y detección de fraudes.

Este enfoque también se usa en librerías como:

XGBoost + Logistic Regression

LightGBM + Logistic Regression

Y es común en producción a gran escala en compañías como Facebook, Alibaba y Tencent.


Hay que tener presente que no se agregarán las nuevas columnas directamente al DataFrame original X_train. 

En cambio, las nuevas características generadas (los índices de hojas codificados como variables binarias) se almacenan en una matriz separada llamada X_one_hot, 

que luego se usa como entrada para entrenar el modelo de regresión logística.

In [6]:
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

gbdt = GradientBoostingClassifier(n_estimators=20)
one_hot = OneHotEncoder()

X_train = X_train[[ 'Pclass', 'Age', 'Fare', 'SibSp']].fillna(0)
X_test = X_test[[ 'Pclass', 'Age', 'Fare', 'SibSp']].fillna(0)

gbdt.fit(X_train, y_train)

X_leaf_index = gbdt.apply(X_train)[:, :, 0]  # apply return the node index on each tree 
print("sample's belonging node of each base tree \n'",X_leaf_index)
# fit one-hot encoder
one_hot.fit(X_leaf_index)   
X_one_hot = one_hot.transform(X_leaf_index)  

sample's belonging node of each base tree 
' [[ 7.  7.  6. ...  4.  7.  4.]
 [ 7.  7.  6. ... 14.  7.  7.]
 [11. 11. 11. ...  4.  6. 11.]
 ...
 [10. 10. 10. ...  4.  6. 10.]
 [13. 14. 13. ...  4.  7. 13.]
 [ 7.  7.  6. ...  6.  7.  7.]]


C:\virtual_environment\bootcamp03\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(


Se generan automáticamente features no lineales y de alta interacción usando las estructuras internas de GBDT, y luego estás entrenando un modelo simple (Regresión Logística) sobre esas features transformadas.

Este enfoque:

Captura relaciones complejas no lineales con GBDT.

Permite interpretabilidad y escalabilidad con la Regresión Logística.

In [7]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='lbfgs', max_iter=1000)
lr.fit(X_one_hot,y_train)
y_pred = lr.predict_proba(
    one_hot.transform(gbdt.apply(X_test)[:, :, 0]))[:,1]
fpr_grd_lm, tpr_grd_lm, _ = roc_curve(y_test, y_pred)
print("AUC for GBDT derived feature + LR：", roc_auc_score(y_test, y_pred))

AUC for GBDT derived feature + LR： 0.7746130952380952


C:\virtual_environment\bootcamp03\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
